In [46]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward_propagation(self, input_arg):
        raise NotImplemented

    def backward_propagation(self, output_error, learning_rate):
        raise NotImplemented

In [47]:
def matrix_multiplication(a, b):
    if isinstance(a[0], int):
        no_rows = 1
        result = [[0 for _ in range(len(b[0]))] for _ in range(no_rows)]
        for i in range(len(result)):
            for j in range(len(result[0])):
                for k in range(len(b[0])):
                    result[i][j] += a[k] * b[k][j]

    else:
        no_rows = len(a)
        result = [[0 for _ in range(len(b[0]))] for _ in range(no_rows)]
        for i in range(len(result)):
            for j in range(len(result[0])):
                for k in range(len(b[0])):
                    result[i][j] += a[i][k] * b[k][j]

    return result


def matrix_add(a, b):
    result = [[0 for _ in range(len(a[0]))] for _ in range(len(a))]
    for i in range(len(a)):
        for j in range(len(a[0])):
            result[i][j] = a[i][j] + b[i][j]

    return result

def transpose_matrix(a):
    result = [[0 for _ in range(len(a))] for _ in range(len(a[0]))]
    for i in range(len(a)):
        for j in range(len(a[0])):
            result[j][i] = a[i][j]

    return result

In [48]:
import random


class FullyConnectedLayer(Layer):

    def __init__(self, input_size, output_size):
        super().__init__()
        self.weights = [[random.random() for _ in range(output_size)] for _ in range(input_size)]
        self.bias = [random.random() for _ in range(output_size)]

    def forward_propagation(self, input_arg):
        self.input = input_arg
        self.output = matrix_add(matrix_multiplication(self.input, self.weights), self.bias)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        input_error = matrix_multiplication(output_error, transpose_matrix(self.weights))
        weights_error = matrix_multiplication(transpose_matrix(self.input), output_error)

        self.weights = [x - learning_rate * weights_error for x in self.weights]
        self.bias = self.bias - learning_rate * output_error
        return input_error

In [49]:
class ActivationLayer(Layer):
    def __init__(self, activation_function, activation_prime):
        super().__init__()
        self.input_data = None
        self.output_data = None
        self.activation = activation_function
        self.activation_prime = activation_prime

    def forward_propagation(self, input_arg):
        self.input_data = input_arg
        self.output_data = self.activation(input_arg)
        return self.output_data

    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [50]:
from math import exp


def relu(x):
    return [max(0, y) for y in x]


def relu_prime(x):
    return [1 for _ in x]


def sigmoid(x):
    return [exp(y) / (exp(y) + 1) for y in x]


def sigmoid_prime(x):
    return [sigmoid(y) * (1 - sigmoid(y)) for y in x]

In [51]:
def mse(yp, yr):
    return sum([(yp[i] - yr[i]) ** 2 for i in range(len(yr))]) / len(yr)


def mse_prime(yp, yr):
    return 2 / len(yr) * sum([(yp[i] - yr[i]) for i in range(len(yr))])


In [52]:
from random import shuffle


class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_arg):
        samples = len(input_arg)
        result = []
        for i in range(samples):
            output = input_arg[i]
            for layer in self.layers:
                layer: Layer
                output = layer.forward_propagation(output)
            result.append(output)
        return result

    def fit(self, train_input_arg, train_output_arg, epochs=500, learning_rate=0.001):
        samples = len(train_input_arg)
        for i in range(epochs):
            err = 0
            train_input_shuffled = random.sample(train_input_arg, len(train_input_arg))
            for j in range(samples):
                output = train_input_shuffled[j]
                for layer in self.layers:
                    layer: Layer
                    output = layer.forward_propagation(output)

                err += self.loss(train_output_arg[j], output)

                error = self.loss_prime(train_output_arg[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)

            err /= samples
            print('epoch %d/%d   error=%f' % (i + 1, epochs, err))

In [53]:
import sqlite3
from PIL import Image
from io import BytesIO


def load_data_from_db(test, target_size=(32, 32)):
    conn = sqlite3.connect('images.db')
    cursor = conn.cursor()
    cursor.execute("SELECT imagine, sepia FROM Imagini WHERE test = ?", (True if test == 1 else False,))
    rows = cursor.fetchall()
    conn.close()
    
    x = []
    y = []
    for row in rows:
        imagine_blob, sepia = row
        imagine_bytes = BytesIO(imagine_blob)
        imagine = Image.open(imagine_bytes)
    
        if imagine.mode == "P":
            imagine = imagine.convert("RGBA")
        if imagine.mode != 'RGB':
            imagine = imagine.convert('RGB')
    
        imagine = imagine.resize(target_size)
    
        imagine_data = list(imagine.getdata())
    
        imagine_list = [component / 255.0 for pixel in imagine_data for component in pixel]
    
        x.append(imagine_list)
        y.append(1 if sepia else 0)
    
    return x, y


In [54]:
train_input, train_output = load_data_from_db(0)
test_input, test_output = load_data_from_db(1)
print(train_input[0])

[0.2549019607843137, 0.16470588235294117, 0.1411764705882353, 0.25882352941176473, 0.16862745098039217, 0.1411764705882353, 0.25882352941176473, 0.17254901960784313, 0.1450980392156863, 0.26666666666666666, 0.1803921568627451, 0.1568627450980392, 0.30980392156862746, 0.20392156862745098, 0.1803921568627451, 0.3215686274509804, 0.21568627450980393, 0.19215686274509805, 0.2549019607843137, 0.20392156862745098, 0.19607843137254902, 0.25098039215686274, 0.21568627450980393, 0.21568627450980393, 0.24313725490196078, 0.21176470588235294, 0.2196078431372549, 0.5882352941176471, 0.5529411764705883, 0.5019607843137255, 0.796078431372549, 0.7333333333333333, 0.6784313725490196, 0.615686274509804, 0.5725490196078431, 0.5333333333333333, 0.23137254901960785, 0.2196078431372549, 0.23137254901960785, 0.4745098039215686, 0.3568627450980392, 0.3607843137254902, 0.6352941176470588, 0.5254901960784314, 0.47058823529411764, 0.37254901960784315, 0.37254901960784315, 0.3568627450980392, 0.29411764705882354

In [55]:
model = Network()

model.add(FullyConnectedLayer(32 * 32, 100))
model.add(FullyConnectedLayer(100, 1))
model.add(ActivationLayer(sigmoid, sigmoid_prime))

model.use(mse, mse_prime)

model.fit(train_input, train_output)

predicted = model.predict(test_input)

TypeError: 'float' object is not subscriptable